# Olympic Webscraping and Data Exploration
#### Ben Wilson

I'll be attempting to scrape the [Database Olympics](www.databaseolympics.com) site to speed up my data collection process. Once that data is cleaned, I'll load it into a local SQLite DB. From there we can explore a few basic metrics and make our first simple model.

In [2]:
from bs4 import BeautifulSoup
import requests

url = 'http://www.databaseolympics.com/games/gamessport.htm?g=47&sp=BAD'
raw_data = requests.get(url)
soup_data = BeautifulSoup(raw_data.content)
raw_table = soup_data('table')[2]

print raw_table

<table border="0" cellpadding="2" cellspacing="0" class="pt8" width="600">
<tr class="statHead">
<td class="cen">Event</td>
<td class="cen">Athlete</td>
<td class="cen">Country</td>
<td class="cen">Result</td>
<td class="cen">Medal</td>
</tr>
<tr class="cl2">
<td>Men's Singles</td>
<td><a href="/players/playerpage.htm?ilkid=DANLIN01  ">Lin Dan</a></td>
<td class="cen"><a href="/country/countryyear.htm?cty=CHN&amp;g=47">CHN</a></td>
<td class="rht"></td>
<td class="cen">GOLD</td>
</tr>
<tr class="cl1">
<td></td>
<td><a href="/players/playerpage.htm?ilkid=ChongLee01">Lee ChongWei</a></td>
<td class="cen"><a href="/country/countryyear.htm?cty=MAS&amp;g=47">MAS</a></td>
<td class="rht"></td>
<td class="cen">SILVER</td>
</tr>
<tr class="cl2">
<td></td>
<td><a href="/players/playerpage.htm?ilkid=JinChe01  ">Chen Jin</a></td>
<td class="cen"><a href="/country/countryyear.htm?cty=CHN&amp;g=47">CHN</a></td>
<td class="rht"></td>
<td class="cen">BRONZE</td>
</tr>
<tr class="cl1">
<td>Men's Doubl

/Users/mrben/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Great! We have the table we want.

I should also point out that the url had two HTTP query variables
- __g=47__: the G stands for the game, which Olympics it is.
- __sp=BAD__: SP is Sport. Then you have to give a sport abbreviation. In this case it was BAD for Badmitton.

Now let's get that table parsed into a dataframe

In [3]:
import pandas
import numpy

col_name = []
for child in raw_table('tr'): 
    if child.get('class', ['Null'])[0] == 'statHead': 
        for td in child.find_all('td'): 
            col_name.append(td.get_text())
        break
results_table = pandas.DataFrame(columns = col_name)

sports_row_num = 0
event_curr = ''

for child in raw_table('tr'):
    row_curr = []
    if (child.get('class', ['Null'])[0] == 'cl1' or child.get('class', ['Null'])[0] == 'cl2'):
        col_num = 1
        for td in child.find_all('td'): 
            if col_num == 1:
                if len(td.get_text()) > 0:
                    event_curr = td.get_text()
                row_curr.append(event_curr)
            else:
                row_curr.append(td.get_text())
            col_num += 1
    if len(row_curr) == len(col_name): 
        results_table.loc[sports_row_num] = row_curr
        sports_row_num += 1
        
        
results_table['Sport'] = 'BAD'
results_table['Game'] = 47

print results_table

              Event                Athlete Country Result   Medal Sport  Game
0     Men's Singles                Lin Dan     CHN           GOLD   BAD    47
1     Men's Singles           Lee ChongWei     MAS         SILVER   BAD    47
2     Men's Singles               Chen Jin     CHN         BRONZE   BAD    47
3     Men's Doubles            Markis Kido     INA           GOLD   BAD    47
4     Men's Doubles        Hendra Setiawan     INA           GOLD   BAD    47
5     Men's Doubles             Fu Haifeng     CHN         SILVER   BAD    47
6     Men's Doubles                Cai Yun     CHN         SILVER   BAD    47
7     Men's Doubles             Lee Jaejin     KOR         BRONZE   BAD    47
8     Men's Doubles            Hwang Jiman     KOR         BRONZE   BAD    47
9   Women's Singles             Zhang Ning     CHN           GOLD   BAD    47
10  Women's Singles           Xie Xingfang     CHN         SILVER   BAD    47
11  Women's Singles  Maria KristinYulianti     INA         BRONZ

Great! This looks like the format we want the data to be in. Let's set it up to look at all the Olympic years and sports.

I've made two reference CSVs. One for the Olypmic ID, year, and host. Then another for all the sports and the abbreviations.

In [4]:
import csv

olympics = pandas.read_csv('olympic_reference.csv')
sports = pandas.read_csv('olympic_sport_reference.csv')
        
# We can loop over all the olympics
# then filter down to the sports for that season (summer or winter)
# and collect all the sports keys to loop over for each of our scraping requests
print sports.sport_key[(sports.season == olympics.season[27])] # the '27' is the olympic loop index

43    ALP
44    BIA
45    BOB
46    CUR
47    FIG
48    ICE
49    LUG
50    NOR
51    SPE
Name: sport_key, dtype: object


Alright, now that we can get the __g__ and __sp__ needed for our request out of the reference tables, our script will grab
- All years
- All sports
- All events
- All medals
- All medal athletes

That is pretty effective compared to manually copy and pasting this information off the site.

Let's create the loop!

In [5]:
results_headers = ['olympic_id', 'sport_key', 'event', 'athlete', 'country', 'result', 'medal']
results_table = pandas.DataFrame(columns = results_headers)

# for index, curr_olympic in olympics.iterrows():
#     curr_sports = sports[(sports.season == curr_olympic.season)]
for i, sport in sports.iterrows():
    url = url = 'http://www.databaseolympics.com/games/gamessport.htm' \
                '?g=30' + \
                '&sp=' + str(sport.sport_key)
#         url = url = 'http://www.databaseolympics.com/games/gamessport.htm' \
#                     '?g=' + str(curr_olympic.olympic_id) + \
#                     '&sp=' + str(sport.sport_key)
        
    raw_data = requests.get(url)
    soup_data = BeautifulSoup(raw_data.content)
    raw_table = soup_data('table')[2]

    sports_row_num = 0
    event_curr = ''

    for child in raw_table('tr'):
        row_curr = [curr_olympic.olympic_id, sport.sport_key]
        if (child.get('class', ['Null'])[0] == 'cl1' or child.get('class', ['Null'])[0] == 'cl2'):
            col_num = 1
            for td in child.find_all('td'): 
                if col_num == 1:
                    if len(td.get_text()) > 0:
                        event_curr = td.get_text()
                    row_curr.append(event_curr)
                else:
                    row_curr.append(td.get_text())
                col_num += 1
        if len(row_curr) == len(col_name): 
            results_table.loc[sports_row_num] = row_curr
            sports_row_num += 1
                
print results_table.head()

NameError: name 'curr_olympic' is not defined

In [14]:
olympic_results_headers = ['olympic_id', 'sport_key', 'event', 'athlete', 'country', 'result', 'medal']
olympic_results_table = pandas.DataFrame(columns = olympic_results_headers)

for i, sport in sports.iterrows():
    curr_url = 'http://www.databaseolympics.com/games/gamessport.htm' \
                '?g=30' + \
                '&sp=' + str(sport.sport_key)
        
    raw_response_data = requests.get(curr_url)
    soup_response_data = BeautifulSoup(raw_response_data.content)
    raw_response_table = soup_response_data('table')[2]

    sport_row_num = 0
    curr_sport_event = ''
    
    if len(raw_response_table('tr')) > 1:
        for child in raw_response_table('tr'):
            curr_row = [olympics.olympic_id[29], sport.sport_key]
            if (child.get('class', ['Null'])[0] == 'cl1' or child.get('class', ['Null'])[0] == 'cl2'):
                col_num = 1
                for td in child.find_all('td'): 
                    if col_num == 1:
                        if len(td.get_text()) > 0:
                            event_curr = td.get_text()
                        curr_row.append(event_curr)
                    else:
                        curr_row.append(td.get_text())
                    col_num += 1
            if len(curr_row) == len(olympic_results_headers): 
                olympic_results_table.loc[sport_row_num] = curr_row
                sport_row_num += 1
        
print olympic_results_table.head()

   olympic_id sport_key      event            athlete country  result   medal
0        30.0       SPE   500m Men    Ivar Ballangrud     NOR    43.4    GOLD
1        30.0       SPE   500m Men         Georg Krog     NOR    43.5  SILVER
2        30.0       SPE   500m Men     Leo Freisinger     USA    44.0  BRONZE
3        30.0       SPE  1500m Men  Charles Mathiesen     NOR  2:19.2    GOLD
4        30.0       SPE  1500m Men    Ivar Ballangrud     NOR  2:20.2  SILVER


In [ ]:
olympic_results_headers = ['olympic_id', 'sport_key', 'event', 'athlete', 'country', 'result', 'medal']
olympic_results_table = pandas.DataFrame(columns = olympic_results_headers)

for index, curr_olympic in olympics.iterrows():
    curr_sports = sports[(sports.season == curr_olympic.season)]
    for i, sport in curr_sports.iterrows():
        curr_url = 'http://www.databaseolympics.com/games/gamessport.htm' \
                    '?g=30' + \
                    '&sp=' + str(sport.sport_key)

        raw_response_data = requests.get(curr_url)
        soup_response_data = BeautifulSoup(raw_response_data.content)
        raw_response_table = soup_response_data('table')[2]

        sport_row_num = 0
        curr_sport_event = ''

        if len(raw_response_table('tr')) > 1:
            for child in raw_response_table('tr'):
                curr_row = [olympics.olympic_id[29], sport.sport_key]
                if (child.get('class', ['Null'])[0] == 'cl1' or child.get('class', ['Null'])[0] == 'cl2'):
                    col_num = 1
                    for td in child.find_all('td'): 
                        if col_num == 1:
                            if len(td.get_text()) > 0:
                                event_curr = td.get_text()
                            curr_row.append(event_curr)
                        else:
                            curr_row.append(td.get_text())
                        col_num += 1
                if len(curr_row) == len(olympic_results_headers): 
                    olympic_results_table.loc[sport_row_num] = curr_row
                    sport_row_num += 1
        
print olympic_results_table.head()